In [1]:
from cobaya.model import get_model
from cobaya.yaml import yaml_load_file

In [8]:
info_cmblens=yaml_load_file('info_yaml_cmblens.yaml')
info_full=yaml_load_file('info_yaml_full.yaml')

In [9]:
model_cmblens=get_model(info_cmblens)
model_full=get_model(info_full)

[model] *WARNING* Ignored blocks/options: ['sampler']
[camb] Importing *local* CAMB from /Users/martina/Documents/University/Codici/cobaya_modules/code/CAMB/
[solike.mflike.mflike] Initialising.
[model] *WARNING* Ignored blocks/options: ['sampler']
[camb] Importing *local* CAMB from /Users/martina/Documents/University/Codici/cobaya_modules/code/CAMB/
[solike.mflike.mflike] Initialising.


There is a difference on how input and sampled parameters are added to model likelihoods if one uses the multi-gaussian framework (full) or piles up individual likelihoods (mflike+lensing, tagged cmblens). In the cmblens case, the sampled parameters are all loaded correctly. In the full case, parameterization.sampled_params() only includes cosmological params (no nuisance). Same as input_params, with the only difference that input_params includes fixed nuisance (fixed in the info_yaml) for the full case. See next cells. I believe this prevents the full case to work correctly in a MCMC. 

In [10]:
model_cmblens.parameterization.sampled_params(), model_full.parameterization.sampled_params()

(OrderedDict([('cosmomc_theta', None),
              ('logA', None),
              ('ns', None),
              ('ombh2', None),
              ('omch2', None),
              ('Alens', None),
              ('tau', None),
              ('a_tSZ', None),
              ('a_kSZ', None),
              ('a_p', None),
              ('beta_p', None),
              ('a_c', None),
              ('beta_c', None),
              ('a_s', None)]),
 OrderedDict([('cosmomc_theta', None),
              ('logA', None),
              ('ns', None),
              ('ombh2', None),
              ('omch2', None),
              ('Alens', None),
              ('tau', None)]))

In [5]:
model_cmblens.parameterization.input_params(), model_full.parameterization.input_params()

(OrderedDict([('cosmomc_theta', None),
              ('As', None),
              ('ns', None),
              ('ombh2', None),
              ('omch2', None),
              ('Alens', None),
              ('tau', None),
              ('n_CIBC', 1.2),
              ('T_d', 9.6),
              ('a_tSZ', None),
              ('a_kSZ', None),
              ('a_p', None),
              ('beta_p', None),
              ('a_c', None),
              ('beta_c', None),
              ('a_s', None)]),
 OrderedDict([('cosmomc_theta', None),
              ('As', None),
              ('ns', None),
              ('ombh2', None),
              ('omch2', None),
              ('Alens', None),
              ('tau', None)]))

If we now look at individual likelihood components, the input_params are different. In the cmblens case, mflike correctly has all nuisance (varying and fixed). In the full case, mflike only has those nuisance that are fixed in the info yaml.

In [6]:
mf_cmblens=model_cmblens.components[0]
lens_cmblens=model_cmblens.components[1]
mf_full=model_full.components[0]
lens_full=model_full.components[1]

In [7]:
mf_cmblens.input_params, mf_full.input_params

(['n_CIBC', 'T_d', 'a_tSZ', 'a_kSZ', 'a_p', 'beta_p', 'a_c', 'beta_c', 'a_s'],
 [])

As far as lensing is concerned, in the cmblens case it does not have any input_params associated (as it should, since there is no nuisance so far directly related to lensing, I believe). In the full case, the input_params has all cosmological params in it.

In [9]:
lens_cmblens.input_params, lens_full.input_params

([], ['cosmomc_theta', 'ombh2', 'omch2', 'Alens', 'tau'])

In [10]:
mf_cmblens.params, mf_full.params

(OrderedDict([('a_tSZ',
               OrderedDict([('prior',
                             OrderedDict([('min', 3.0), ('max', 3.6)])),
                            ('proposal', 0.05),
                            ('latex', 'a_\\mathrm{tSZ}')])),
              ('a_kSZ',
               OrderedDict([('prior',
                             OrderedDict([('min', 1.4), ('max', 1.8)])),
                            ('proposal', 0.1),
                            ('latex', 'a_\\mathrm{kSZ}')])),
              ('a_p',
               OrderedDict([('prior',
                             OrderedDict([('min', 6.2), ('max', 7.6)])),
                            ('proposal', 0.075),
                            ('latex', 'a_p')])),
              ('beta_p',
               OrderedDict([('prior',
                             OrderedDict([('min', 1.8), ('max', 2.2)])),
                            ('proposal', 0.015),
                            ('latex', '\\beta_p')])),
              ('a_c',
               Ordere

In [11]:
model_cmblens.input_params, model_full.input_params

(['cosmomc_theta',
  'As',
  'ns',
  'ombh2',
  'omch2',
  'Alens',
  'tau',
  'n_CIBC',
  'T_d',
  'a_tSZ',
  'a_kSZ',
  'a_p',
  'beta_p',
  'a_c',
  'beta_c',
  'a_s'],
 ['cosmomc_theta',
  'As',
  'ns',
  'ombh2',
  'omch2',
  'Alens',
  'tau',
  'n_CIBC',
  'T_d'])

If I now access the input_params argument of the multi-gaussian likelihood, I should see all input params for all likelihoods (I guess), i.e. all nuisance. Instead, I only see those nuisance fixed in the info yaml (the same nuisance I see if I access the input_params argument of the component mflike). I thought I should see the sum of all input_params of all components (mflike and lensing), but I only see those form mflike. 

In [12]:
multig=model_full.components[0]
multig

solike.gaussian.MultiGaussianLikelihood

In [13]:
multig.input_params, mf_full.input_params, lens_full.input_params

(['n_CIBC', 'T_d'],
 ['n_CIBC', 'T_d'],
 ['cosmomc_theta', 'ombh2', 'omch2', 'Alens', 'tau'])

Let's now try to compute the loglike. I define a point:

In [14]:
point=({'cosmomc_theta': 0.010494998852840933, 'logA': 3.162264731376158, 'ns': 0.9932413477950633, 
        'ombh2': 0.017367921092627116, 'omch2': 0.14101557864954728, 'Alens': 0.6964645133740484, 
        'tau': 0.05364711120741492, 'a_tSZ': 3.350883132982168, 'a_kSZ': 1.664578437073561, 
        'a_p': 7.06446968084832, 'beta_p': 2.1410226683881706, 'a_c': 4.7968238875183, 
        'beta_c': 2.2734481216978546, 'a_s': 3.384387972460651})

And compute the loglikes in both the cmblens case (individual loglikes for mflike and lensing, followed by values of derived params; full loglike followed by derived params):

In [15]:
model_cmblens.loglikes(point), model_cmblens.loglike(point)

((array([-15506.37892786,   -489.376359  ]),
  [2.3624037489771233e-09, 59.507384937062774, 0.9602662296773381]),
 (-15995.755286864982,
  [2.3624037489771233e-09, 59.507384937062774, 0.9602662296773381]))

And for the full case (you see it fails):

In [16]:
model_full.loglikes(point)

[parameterization] *ERROR* Unknown parameter 'a_tSZ'.


LoggedError: Unknown parameter 'a_tSZ'.

In [17]:
model_full.loglike(point)

[parameterization] *ERROR* Unknown parameter 'a_tSZ'.


LoggedError: Unknown parameter 'a_tSZ'.

The full case fails, since (as we have seen above), the sampled nuisance params are not correctly loaded in the model. I must enter the nuisance params at the info yaml level, as done in e.g. https://github.com/simonsobs/Likelihoods/blob/master/notebooks/multi-like-test.ipynb. However, we would like not to be doing this, as we ideally want to 'hide' nuisance params in the individual likelihood yaml.
I also tried adding the MFLike.yaml to the options argument of the likelihood class in the info yaml for the full case, but this does not work either.

In [18]:
model_full.info()['likelihood']

OrderedDict([('solike.gaussian.MultiGaussianLikelihood',
              OrderedDict([('speed', -1),
                           ('stop_at_error', False),
                           ('version', None),
                           ('name', 'Gaussian'),
                           ('datapath', None),
                           ('covpath', None),
                           ('components',
                            ['solike.mflike.MFLike',
                             'solike.SimulatedLensingLikelihood']),
                           ('options',
                            [OrderedDict([('path',
                                           '/Users/martina/Documents/University/Projects/solike_sandbox/software/solike/'),
                                          ('data_folder',
                                           '/Users/martina/Documents/University/Projects/solike_sandbox/modules/data/LAT_MFLike_data/like_products'),
                                          ('sim_id', 0),
                  

In [53]:
model_cmblens.info()['likelihood']

OrderedDict([('solike.mflike.MFLike',
              OrderedDict([('speed', -1),
                           ('stop_at_error', False),
                           ('version', None),
                           ('name', 'TT'),
                           ('datapath', None),
                           ('covpath', None),
                           ('kind', 'tt'),
                           ('lmax', 6002),
                           ('path',
                            '/Users/martina/Documents/University/Projects/solike_sandbox/software/solike/'),
                           ('data_folder',
                            '/Users/martina/Documents/University/Projects/solike_sandbox/modules/data/LAT_MFLike_data/like_products'),
                           ('sim_id', 0),
                           ('select', 'tt-te-ee'),
                           ('experiments',
                            [OrderedDict([('LAT', [93, 145, 225])])]),
                           ('foregrounds',
                          